# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Управление браузером с Selenium: продолжение

## Часть 1: пример скроллинга с Selenium 

Импортируем все необходимые модули и функции:

In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from getpass import getpass
from bs4 import BeautifulSoup
from time import sleep

Сразу запрашиваем и сохраняем свой пароль от ВКонтакте:

In [2]:
my_password = getpass()

········


Функция для извлечения имен пользователей и ссылок на их аккаунты (с прошлого практикума):

In [3]:
def get_person(d):
    name = d.text
    href = d.find("a")["href"]
    full_href = "https://vk.com" + href
    return name, full_href

Запускаем браузер через Selenium:

In [5]:
br = wd.Chrome(executable_path=r'D:\Загрузки\chromedriver_win32 (1)\chromedriver.exe')

<ipython-input-5-10feb1d0acbf>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  br = wd.Chrome(executable_path=r'D:\Загрузки\chromedriver_win32 (1)\chromedriver.exe')


Находим поле для логина и заполняем:

In [ ]:
br.get("https://vk.com/")
br.implicitly_wait(3)

In [6]:
login = br.find_element(By.ID, "index_email")
login.send_keys("allatambov@mail.ru")
br.implicitly_wait(3)
login.send_keys(Keys.ENTER)

Находим поле для пароля и заполняем (к слову, `Keys.ENTER` и `Keys.RETURN` – одно и то же, одна и та жа клавиша для перехода на новую строку):

In [7]:
password = br.find_element(By.NAME, "password")
password.send_keys(my_password)
br.implicitly_wait(3)
password.send_keys(Keys.RETURN)

Переходим на страницу для поиска людей:

In [8]:
br.get("https://vk.com/friends")

Выставляем все фильтры для поиска сразу (код с прошлого практикума, только в одной ячейке):

In [9]:
search = br.find_element(By.LINK_TEXT, "Поиск друзей")
search.click() 
br.implicitly_wait(2)

pars = br.find_element(By.ID, "friends_filters_block")
pars.click()
br.implicitly_wait(2)

city = br.find_element(By.ID, "cCity")
city_inp = city.find_element(By.TAG_NAME, "input")
city_inp.send_keys("Москва") 
br.implicitly_wait(3)
city_inp.send_keys(Keys.RETURN)

sex = br.find_element(By.ID, "cSex")
values = sex.find_elements(By.TAG_NAME, "div")
values[1].click()

Извлекаем исходный код страницы и вспоминаем, как у нас извлекалась информация о пользователях в прошлый раз:

In [10]:
html = br.page_source
soup = BeautifulSoup(html)
divs = soup.find_all("div", {"class" : "labeled name"})
L = [get_person(d) for d in divs] 

In [11]:
L

[('Дмитрий Сорокин', 'https://vk.com/birdnice'),
 ('Никита Зарипов', 'https://vk.com/letsgowithyou'),
 ('Мишаня Аникин', 'https://vk.com/muaymikethai'),
 ('Владислав Рубанов', 'https://vk.com/rubanoww'),
 ('Кирилл Петров', 'https://vk.com/warpath031'),
 ('Сергей Проценко', 'https://vk.com/prsevld'),
 ('Иоанн Довгополый', 'https://vk.com/dereinzigemitdemeigentum'),
 ('Степан Кyзнeцoв', 'https://vk.com/id104687424'),
 ('Никита Бубнов', 'https://vk.com/mrnicknoker'),
 ('Стас Васильев', 'https://vk.com/stas_satori'),
 ('Артём Черняев-Ермоленко', 'https://vk.com/tatorhe'),
 ('Егор Гладких', 'https://vk.com/yegomaru'),
 ('Илья Иншаков', 'https://vk.com/id136161093'),
 ('Руслан Гончар', 'https://vk.com/ruslhu'),
 ('Илья Демидов', 'https://vk.com/lyas1k'),
 ('Даниил Аксёнов', 'https://vk.com/danhse'),
 ('Артем Терещенков', 'https://vk.com/tereshchka'),
 ('Михаил Садов', 'https://vk.com/m_sadov'),
 ('Азиз Тулаганов', 'https://vk.com/aziz_tulaganov'),
 ('Рустам Бафоев', 'https://vk.com/id3313822

Осталась одна проблема – мы извлекли далеко не все результаты, а только те результаты, которые «видит» браузер, то есть то, что мы как пользователи видим до скроллинга. Библиотека `selenium` умеет скроллить страницы, точнее, активировать запуск кода на JavaScript, который отвечает за скроллинг. В общем виде строка с кодом для скроллинга выглядит так (`Y` – на сколько пикселей нужно проскроллить):

    br.execute_script("window.scrollTo(0, Y)") 
    
Если нужно проскроллить до конца страницы, то тогда вместо `Y` нужно вписать значение, которое извлекается из тела документа HTML:

     document.body.scrollHeight

Например, проскроллим текущую страницу до самого низа:

In [12]:
br.execute_script("window.scrollTo(0, document.body.scrollHeight);")

Далее, чтобы открыть новые результаты, нам нужно найти кнопку *Показать ещё*. Если посмотрим внимательно на исходный код, найдём id этой кнопки:

In [13]:
button_more = br.find_element(By.ID, "ui_search_load_more")

Кликаем:

In [14]:
button_more.click()

Теперь нам нужно сделать следующее: выгружать информацию из исходного кода страницы по новым пользователям и скроллить страницу дальше. И повторять эти действия до тех пор, пока результаты не закончатся. Для начала напишем функцию `get_users_info()`, которая принимает на вход объект `br` (окно браузера), считывает исходный код и возвращает список с «чистыми» результатами – список пар *(имя пользователя, ссылка на профиль)*.

### Задача 1

Напишите функцию `get_users_info()` согласно описанию выше.

In [15]:
def get_users_info(br):
    html = br.page_source
    soup = BeautifulSoup(html)
    divs = soup.find_all("div", {"class" : "labeled name"})
    L = [get_person(d) for d in divs] 
    return L

Теперь напишем код (адаптированный ответ с [StackOverflow](https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python)) для цикла. Какой тип цикла нам нужен? Нам нужен цикл, который умеет повторять операции до тех пор, пока мы не дойдём до последнего результата, то есть до того момента, когда скроллить будет некуда. Воспользуемся конструкцией `while True`, бесконечным вариантом цикла `while`, который будет запускаться до тех пор, пока не дойдёт до кода с оператором `break` (выход из цикла) или не столкнётся с ошибкой. 

In [16]:
# результатов много, фильтры очень общие
# оставновим цикл сами через Стоп (Kernel – Interrupt)

all_results = []
last_height = br.execute_script("return document.body.scrollHeight")

while True:
    res = get_users_info(br)
    all_results.extend(res) 
    
    br.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    new_height = br.execute_script("return document.body.scrollHeight")
    sleep(1)
    
    if new_height == last_height:
        break
        
    last_height = new_height

KeyboardInterrupt: 

Пояснения к коду.

1. Сохраняем в переменную `last_height` величину, на которую мы можем проскроллить страницу за один раз в данный момент времени, чтобы дойти до конца страницы. 

2. На каждой итерации цикла `while` мы выгружаем информацию, добавляем её в список `all_results` и скроллим страницу до самого низа. После скроллинга проверяем, на сколько ещё можно проскроллить, сохраняем полученное значение в `new_height`. 

3. Если скроллить уже некуда, если мы находимся в самом низу «бесконечной» страницы с результатами поиска, то `new_height` совпадает с `last_height`. Значит, нам нужно остановить исполнение кода – выходим из цикла с помощью `break`. 

4. Если мы ещё не закончили скроллить, обновляем значение `last_height`, заменяя его на `new_height` (теперь уже в нём хранится величина, на которую мы можем проскроллить страницу за один раз в данный момент времени). Продолжаем выполнять выгрузку информации и скроллинг.

### Задача 2

Создайте новый список `results_unique`, который не содержит повторений (набор уникальных пар значений).

In [19]:
results_unique = list(set(all_results))

### Задача 3

Создайте датафрейм на основе списка `results_unique`, присвойте столбцам подходящие названия и выгрузите таблицу в файл Excel.

In [23]:
import pandas as pd
df = pd.DataFrame(results_unique)
df.columns = ["user", "link"]
df.to_excel("vk-users.xlsx")

## Часть 2: выгрузка информации из ссылок в адресной строке

### Задача 1

Напишите функцию `get_link()`, которая принимает на вход строку с адресом, заходит на страницу [Google Maps](https://www.google.com/maps), вводит в поле для поиска этот адрес и сохраняет ссылку, которая скрывает в себе координаты, соответствующие этому адресу.

In [ ]:
# пока работаем с одним примером

br.get("https://www.google.com/maps")

In [25]:
# ищем поле для поиска

search = br.find_element(By.ID, "searchboxinput")
search = br.find_element(By.CSS_SELECTOR, "#searchboxinput")

In [29]:
# вводим адрес – предварительно очищаем поле на всякий случай

addr = "Москва, Мясницкая улица, 20"
search.clear()
search.send_keys(addr)

# ищем кнопку для поиска
button = br.find_element(By.CSS_SELECTOR,"#searchbox-searchbutton")
button.click() 

In [30]:
# сама страница с результатами не нужна
# широта и долгота хранятся в ссылке в адресной строке

br.current_url

'https://www.google.com/maps/place/%D0%9C%D1%8F%D1%81%D0%BD%D0%B8%D1%86%D0%BA%D0%B0%D1%8F+%D1%83%D0%BB.,+20,+%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0,+101000/@55.7615846,37.631129,17z/data=!3m1!4b1!4m5!3m4!1s0x46b54a5de2123c87:0x49e0282321a5956a!8m2!3d55.7615816!4d37.633323'

In [32]:
# наконец, все в одной функции

def get_link(addr):
    search = br.find_element(By.ID, "searchboxinput")
    search.clear()
    search.send_keys(addr)
    sleep(3)
    button = br.find_element(By.CSS_SELECTOR,"#searchbox-searchbutton")
    button.click() 
    sleep(3)
    link = br.current_url
    return link

### Задача 2

Примените эту функцию для всех адресов в списке `addresses` и создайте словарь с парами, где ключом является адрес, а значением – ссылка с координатами. Преобразуйте полученный словарь в датафрейм. 

In [33]:
addresses = ["BROADWAY & LEE ST, Cambridge, MA",
            "100 MAGEE ST, Cambridge, MA",
            "100 Elm St, Cambridge, MA",
            "ATHENAEUM ST & Athenaeum St, Cambridge, MA",
            "HARVARD ST & WINDSOR ST, Cambridge, MA",
            "Massachusetts Ave & Brookline St, Cambridge, MA",
            "PEARL ST & Perry St, Cambridge, MA",
            "Temple St, Cambridge, MA",
            "100 LAWN ST, Cambridge, MA",
            "200 BINNEY ST, Cambridge, MA"]

info = {}
for a in addresses:
    info[a] = get_link(a)
    print(a, addresses.index(a))

BROADWAY & LEE ST, Cambridge, MA 0
100 MAGEE ST, Cambridge, MA 1
100 Elm St, Cambridge, MA 2
ATHENAEUM ST & Athenaeum St, Cambridge, MA 3
HARVARD ST & WINDSOR ST, Cambridge, MA 4
Massachusetts Ave & Brookline St, Cambridge, MA 5
PEARL ST & Perry St, Cambridge, MA 6
Temple St, Cambridge, MA 7
100 LAWN ST, Cambridge, MA 8
200 BINNEY ST, Cambridge, MA 9


### Задача 3

Добавьте в полученный датафрейм столбцы с широтой и долготой, соответствующими каждому адресу.

In [38]:
dat = pd.DataFrame(info.items())

In [45]:
# как разбить ссылку и забрать широту и долготу
# разбиваем по @, забираем последнюю часть

dat[1][0].split("@")

['https://www.google.com/maps/place/Broadway+',
 '+Lee+St/',
 '42.3707009,-71.106863,17z/data=!3m1!4b1!4m5!3m4!1s0x89e3774e15ebc1f7:0xe356728a459fd20e!8m2!3d42.370697!4d-71.104669']

In [49]:
# последнюю часть разбиваем по запятой
# первый элемент – широта
# второй элемент – долгота

dat["lat"] = dat[1].apply(lambda x: x.split("@")[-1].split(",")[0])
dat["long"] = dat[1].apply(lambda x: x.split("@")[-1].split(",")[1])

In [51]:
# переименовываем столбцы

dat.rename(columns = {0: "address", 1 : "link"}, inplace = True)

In [52]:
dat

,address,link,lat,long
0,"BROADWAY & LEE ST, Cambridge, MA",https://www.google.com/maps/place/Broadway+@+L...,42.3707009,-71.106863
1,"100 MAGEE ST, Cambridge, MA",https://www.google.com/maps/place/100+Magee+St...,42.3653226,-71.1137659
2,"100 Elm St, Cambridge, MA","https://www.google.com/maps/place/100+Elm+St,+...",42.3693695,-71.0994694
3,"ATHENAEUM ST & Athenaeum St, Cambridge, MA",https://www.google.com/maps/place/Athenaeum+St...,42.3642168,-71.0834516
4,"HARVARD ST & WINDSOR ST, Cambridge, MA",https://www.google.com/maps/place/Windsor+St+%...,42.366192,-71.0979686
5,"Massachusetts Ave & Brookline St, Cambridge, MA",https://www.google.com/maps/place/Massachusett...,42.3640341,-71.1035858
6,"PEARL ST & Perry St, Cambridge, MA",https://www.google.com/maps/place/Perry+St+%26...,42.3616089,-71.1081985
7,"Temple St, Cambridge, MA","https://www.google.com/maps/place/Temple+St,+C...",42.3663827,-71.1064264
8,"100 LAWN ST, Cambridge, MA","https://www.google.com/maps/place/100+Lawn+St,...",42.37813,-71.1562314
9,"200 BINNEY ST, Cambridge, MA",https://www.google.com/maps/place/200+Binney+S...,42.3656417,-71.0864745
